In [8]:
import numpy as np
import cvxpy as cp

In [9]:
with open('A_21.npy', 'rb') as f:
    A_ = np.load(f) # matrix corresponding to 13th day's travel pattern

with open('D.npy', 'rb') as f:
    D = np.load(f)

In [10]:
A = np.zeros((A_.shape[0],A_.shape[0],int(A_.shape[2])))

for k in range(A.shape[2]):
    A[:,:,k] = np.sum(A_[:,:,k:k+1], axis = 2)

In [11]:
# number of zones
N = D.shape[0]

# number of time slots
K = A.shape[-1]

In [12]:
average_speed = 40 #kmph

T = np.divide(D, average_speed/60) # travel time matrix in minutes

T = np.round(T).astype(int)

for i in range(N):
    T[i,i] = 1
    
T = np.ceil(T).astype(int)

In [13]:
per_km_charge = 2.5/1.60934 # in $
fixed_charge  = 2.5         # in $

p  = D*per_km_charge + fixed_charge


In [14]:
d = 1-D/np.max(D)

In [15]:
V = np.zeros((N,K+50))

# number of taxis
C = 500

for c in range(C):
    V[np.random.randint(N),0] += 1

In [16]:
def scheduling_state_update(U,k):
    for i in range(N):
        for j in range(N):
            V[j, k+T[i,j]] += U[i,j] 
    
    V[:,k] = V[:,k] - np.sum(U,1)
    
def recommendation_state_update(U,k):
    for i in range(N):
        for j in range(N):
            V[j, k+T[i,j]] += U[i,j] 

def wait(k):
    V[:,k+1] += V[:,k] 
    
def scheduling_envionment(k):
    x = cp.Variable((N, N),nonneg=True)
    
    cons = [x <= A[:,:,k], cp.sum(x,1) <= V[:,k]]

    obj  = cp.Maximize(cp.sum(x))

    prob = cp.Problem(obj, cons)

    prob.solve()
        
    scheduling_state_update(x.value, k)

    served_demand.append(np.sum(x.value))

def recommendation_envionment(k,w):
    r = np.sum(A[:,:,k:k+w], axis = (2,1))
    v = np.sum(V[:,k:k+w], axis = 1)
        
    y = cp.Variable((N, N),nonneg=True)
    z = cp.Variable(N,nonneg=True)
    
    cons = [cp.sum(y,1) <= V[:,k], z <= r, z <= v + cp.sum(y,0), cp.multiply(y,T-w)<=0]

    obj  = cp.Maximize(cp.sum(z) + cp.sum(cp.multiply(y,d)))

    prob = cp.Problem(obj, cons)

    prob.solve()
        
    recommendation_state_update(y.value, k)
    
    recommendations.append(np.sum(y.value) - np.trace(y.value))


In [17]:
w = 10 #horizon window
    
V = np.zeros((N,K+50))

# number of taxis
C = 500

for c in range(C):
    V[np.random.randint(N),0] += 1

k = 0

served_demand   = []
recommendations = []

while( k < A.shape[2]):

    scheduling_envionment(k)
    #wait(k)
    recommendation_envionment(k,w)

    k += 1

print('served demand ratio', np.sum(served_demand/np.sum(A)))